In [0]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns

### Загрузка и предобработка входных данных

In [0]:
# Загрузка датасета с Google Drive
df=pd.read_csv('/content/drive/My Drive/Kurs_project/train.csv')
df.head(3)

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,0,40.0,0.0,1400.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,63.0
1,1,48.0,4.0,2850.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,86.0
2,2,39.0,0.0,1200.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0


In [0]:
df.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.878000,1.986800,1699.105000,1.719500,0.375000,0.132900,0.109600,0.053700,0.032100,0.019400,64.340800
std,2886.89568,8.043929,1.772213,524.886654,0.792264,0.484147,0.339484,0.312406,0.225436,0.176274,0.137933,13.536823
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55.000000
50%,4999.50000,46.000000,2.000000,1500.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.000000
max,9999.00000,68.000000,10.000000,3950.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000


In [0]:
# Перечислим признаки и целевую переменную
features = ['age', 'years_of_experience', 'lesson_price', 'qualification', 'physics', 'chemistry', 'biology', 'english', 'geography', 'history']
target = 'mean_exam_points'

In [0]:
# Создадим наборы признаков и целевой переменной
X = df[features].values
y = df[target].values

### Реализуем алгоритм градиентного бустинга

In [0]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [0]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        
        # Расчет среднего значения в листе
        prediction = np.mean(self.labels)
        return prediction        

In [0]:
# Реализуем класс дерева
class Tree:

    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.tree = None


    # В случае регрессии разброс будет характеризоваться дисперсией
    def variance(self, labels):
        return np.var(labels)


    # Расчет качества
    def quality(self, left_labels, right_labels, current_variance):

        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
        
        return current_variance - p * self.variance(left_labels) - (1 - p) * self.variance(right_labels)

    
    # Разбиение датасета в узле
    def split(self, data, labels, index, t):
    
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)
            
        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]
            
        return true_data, false_data, true_labels, false_labels

    
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels):
        
        #  обозначим минимальное количество объектов в узле
        min_leaf = 5

        current_variance = self.variance(labels)

        best_quality = 0
        best_t = None
        best_index = None
        
        n_features = data.shape[1]
        
        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])
            
            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue
                
                current_quality = self.quality(true_labels, false_labels, current_variance)
                
                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
    

    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels, max_depth, depth=0):

        quality, t, index = self.find_best_split(data, labels)

        # Прекращаем рекурсию, когда нет прироста в качества или достигнута
        # максимальная глубина дерева
        if quality == 0 or depth >= max_depth:
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels, max_depth, depth+1)
        false_branch = self.build_tree(false_data, false_labels, max_depth, depth+1)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    

    def predict_target(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.predict_target(obj, node.true_branch)
        else:
            return self.predict_target(obj, node.false_branch)
    

    def predict(self, data):
    
        values = []
        for obj in data:
            prediction = self.predict_target(obj, self.tree)
            values.append(prediction)
        return values
    

    # Строим дерево
    def fit(self, data, labels):
        self.tree = self.build_tree(data, labels, self.max_depth)
        return self

In [0]:
# Реализуем класс для градиентного бустинга
class GB:
    def __init__(self, n_trees, max_depth, coefs, eta):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.coefs = coefs
        self.eta = eta
        self.trees = []


    def bias(self, y, z):
        return (y - z)

    def fit(self, X_train, y_train):
        
        # Деревья будем записывать в список
        trees = []

        for i in range(self.n_trees):
            tree = Tree(max_depth=self.max_depth)         

            # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
            # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
            if len(self.trees) == 0:
                # обучаем первое дерево на обучающей выборке
                tree.fit(X_train, y_train)
            else:
                # Получим ответы на текущей композиции
                target = self.predict(X_train)
                
                # алгоритмы начиная со второго обучаем на сдвиг
                tree.fit(X_train, self.bias(y_train, target))
            self.trees.append(tree)
            
        return self
    
    
    def predict(self, X):
        # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
        # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
        return np.array([sum([self.eta * coef * alg.predict([x])[0] for alg, coef in zip(self.trees, self.coefs)]) for x in X])

In [0]:
# Функция вычисления метрики R2
def R2(y_pred, y_true):
  numerator = ((y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
  denominator = ((y_true - np.average(y_true)) ** 2).sum(axis=0, dtype=np.float64)
  return 1 - (numerator / denominator)

### Проверка модели

In [0]:
# Разобьем тренировочную выборку на две
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [0]:
# Число деревьев в ансамбле
n_trees = 17

# для простоты примем коэффициенты равными 1
coefs = [1] * n_trees

# Максимальная глубина деревьев
max_depth = 5

# Шаг
eta = 0.4

In [0]:
gb = GB(n_trees, max_depth, coefs, eta)
# Проведем обучение модели
gb.fit(X_train, y_train)
# Предскажем значения на тестовом наборе
y_pred_test = gb.predict(X_test)

In [0]:
# Посмотрим значение коэффициента детерминации
R2(y_pred_test, y_test)

0.7787152569386782

n_trees = 17, max_depth = 5, eta = 0.4, r2 = 0.7787152569386782

### Обучим модель на всем наборе тренировочных данных и спрогнозируем значения для тестового набора

In [0]:
# Обучим модель
gb_final = GB(n_trees, max_depth, coefs, eta)
gb_final.fit(X, y)

In [0]:
# Загрузка тестовый датасета с Google Drive
df_test = pd.read_csv('/content/drive/My Drive/Kurs_project/test.csv')
df_test.head(3)

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history
0,10000,46.0,3.0,1050.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,10001,43.0,3.0,1850.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,52.0,1.0,1550.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# Создадим набор признаков
X_test = df_test[features].values

In [0]:

y_pred_test = gb_final.predict(X_test)

### Выгрузим результаты в файл

In [0]:
submissions = pd.concat([df_test['Id'], pd.Series(y_pred_test)], axis=1)
submissions = submissions.rename(columns={0: 'mean_exam_points'})

In [0]:
submissions.to_csv('/content/drive/My Drive/Kurs_project/YRiabinin_predictions.csv',index=None)